In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch.utils.data import random_split, DataLoader
import os

from corner_dataset import CornersDatasetOrdered, CornersDatasetTopToBottom
from corner_detector_linear import CornerDetectorLinear

In [ ]:
from torch.utils.data import random_split

img_dir = "images"
corners_dir = "corners"

full_dataset = CornersDatasetOrdered(
    img_dir=img_dir,
    corners_dir=corners_dir
)

val_ratio = 0.2
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

print(len(train_dataset))
print(len(val_dataset))


In [ ]:
from torch.utils.data import random_split

img_dir = "images"
corners_dir = "corners"

full_dataset = CornersDatasetTopToBottom(
    img_dir=img_dir,
    corners_dir=corners_dir
)

val_ratio = 0.2
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

print(len(train_dataset))
print(len(val_dataset))


809
202


In [4]:
def train(
        train_loader,
        val_loader,
        model,
        num_epochs=30,
        lr=1e-3,
        save_path="corner_detector.pth",
        load_path=None,
):
    save_path = os.path.join("models", save_path)
    os.makedirs("models", exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if load_path is not None:
        load_path = os.path.join("models", load_path)
        model.load_state_dict(torch.load(load_path))

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float("inf")

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0

        for images, targets in train_loader:
            images = images.to(device)
            targets = targets.to(device)

            preds = model(images)

            loss = criterion(preds, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)

        if val_loader is not None:
            model.eval()
            val_running_loss = 0.0

            with torch.no_grad():
                for images, targets in val_loader:
                    images = images.to(device)
                    targets = targets.to(device)

                    preds = model(images)

                    loss = criterion(preds, targets)
                    val_running_loss += loss.item() * images.size(0)

            epoch_val_loss = val_running_loss / len(val_loader.dataset)

            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                torch.save(model.state_dict(), save_path)

            print(
                f"Epoch [{epoch}/{num_epochs}] "
                f"Train Loss: {epoch_train_loss:.6f}  "
                f"Val Loss: {epoch_val_loss:.6f}"
            )
        else:
            print(
                f"Epoch [{epoch}/{num_epochs}] "
                f"Train Loss: {epoch_train_loss:.6f}"
            )

    if val_loader is None:
        torch.save(model.state_dict(), save_path)

    print("Training finished. Best model saved to:", save_path)
    return model

In [5]:
model = CornerDetectorLinear()

In [6]:
trained_model = train(
    train_loader=train_loader,
    val_loader=val_loader,
    model=model,
    num_epochs=80,
    lr=1e-3,
    save_path="corner_detector_v2_test1.pth"
)

Epoch [1/80] Train Loss: 0.160313  Val Loss: 2.719723
Epoch [2/80] Train Loss: 0.009574  Val Loss: 0.007714
Epoch [3/80] Train Loss: 0.005255  Val Loss: 0.004241
Epoch [4/80] Train Loss: 0.005945  Val Loss: 0.006333
Epoch [5/80] Train Loss: 0.004123  Val Loss: 0.002225
Epoch [6/80] Train Loss: 0.003779  Val Loss: 0.002057
Epoch [7/80] Train Loss: 0.002722  Val Loss: 0.002009
Epoch [8/80] Train Loss: 0.002644  Val Loss: 0.001698
Epoch [9/80] Train Loss: 0.001767  Val Loss: 0.001286
Epoch [10/80] Train Loss: 0.002101  Val Loss: 0.002311
Epoch [11/80] Train Loss: 0.002572  Val Loss: 0.001643
Epoch [12/80] Train Loss: 0.002206  Val Loss: 0.001976
Epoch [13/80] Train Loss: 0.002412  Val Loss: 0.001754
Epoch [14/80] Train Loss: 0.002644  Val Loss: 0.001343
Epoch [15/80] Train Loss: 0.001888  Val Loss: 0.002148
Epoch [16/80] Train Loss: 0.002200  Val Loss: 0.002099
Epoch [17/80] Train Loss: 0.001722  Val Loss: 0.001113
Epoch [18/80] Train Loss: 0.001377  Val Loss: 0.001607
Epoch [19/80] Train

In [8]:
trained_model_2 = train(
    train_loader=train_loader,
    val_loader=val_loader,
    model=model,
    num_epochs=150,
    lr=1e-3,
    save_path="corner_detector_test2.pth",
    load_path="corner_detector_test1.pth"
)

Epoch [1/150] Train Loss: 0.127071  Val Loss: 1.026602
Epoch [2/150] Train Loss: 0.040275  Val Loss: 0.175476
Epoch [3/150] Train Loss: 0.028663  Val Loss: 0.073614
Epoch [4/150] Train Loss: 0.029833  Val Loss: 0.031058
Epoch [5/150] Train Loss: 0.018874  Val Loss: 0.025015
Epoch [6/150] Train Loss: 0.018892  Val Loss: 0.019912
Epoch [7/150] Train Loss: 0.015256  Val Loss: 0.028113
Epoch [8/150] Train Loss: 0.024035  Val Loss: 0.031479
Epoch [9/150] Train Loss: 0.024366  Val Loss: 0.032811
Epoch [10/150] Train Loss: 0.016035  Val Loss: 0.020938
Epoch [11/150] Train Loss: 0.010917  Val Loss: 0.013739
Epoch [12/150] Train Loss: 0.012027  Val Loss: 0.020989
Epoch [13/150] Train Loss: 0.008744  Val Loss: 0.014401
Epoch [14/150] Train Loss: 0.023727  Val Loss: 0.020170
Epoch [15/150] Train Loss: 0.008111  Val Loss: 0.016971
Epoch [16/150] Train Loss: 0.006731  Val Loss: 0.017878
Epoch [17/150] Train Loss: 0.011061  Val Loss: 0.013041
Epoch [18/150] Train Loss: 0.006097  Val Loss: 0.017885
E